In [ ]:
#| default_exp fractionalfactorial

## Fractional Factorial Design

In [ ]:
#| export

from pprint import pprint
import numpy as np
import pandas as pd

import fastcore.test as test

from fastDOE.fullfactorial import fullfactorial, fullfactorial2k

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
def handle_generators(n: int, p: int, generators: dict) -> dict:
    """checks and processes the given generators"""
    if len(generators) != p:
        raise ValueError("number of generators has to be the same as p")
    return generators

In [ ]:
#| hide
generator = {6: [3, 4, 5], 7: [1, 2, 4, 5], 8: [1, 2, 3, 5]}
test.test_fail(
    handle_generators,
    msg="number of generators has to be the same as p",
    args=[8, 4, generator]
)

In [ ]:
#| export
def apply_generators(
    model_matrix: np.ndarray, # base factorial model matrix
    generators: dict, # dictionary of generators
) -> np.ndarray:
    """applies the given generators to the base factorial model matrix"""
    matrix = np.insert(model_matrix, 0, 0, axis=1)
    for factor, gen in generators.items():
        new_col = np.product(matrix[:, gen], axis=1).reshape((matrix.shape[0], -1))
        matrix = np.append(matrix, new_col, axis=1)
    return matrix[:, 1:]

**Example**

In [ ]:
apply_generators(fullfactorial2k(2), {3: [1, 2]})

array([[-1, -1,  1],
       [ 1, -1, -1],
       [-1,  1, -1],
       [ 1,  1,  1]])

## fractional factorial designs with 2 levels

function to create a model matrix for a fractional factorial design for n factors with 2 levels each, with n-p runs in standard order

In [ ]:
#| export
def fractionalfactorial2k(
    n: int, # number of factors
    p: int, # number of confounded factors
    generators: dict, # {4: [1, 2, 3]} dictionary of the generators for the confounded factors
    
) -> np.ndarray:
    """build the base model matrix of a 2^(n-p)^ fractional factorial design using the given generators."""
    generators = handle_generators(n, p, generators)
    matrix = fullfactorial2k(n - p)
    matrix = apply_generators(matrix, generators)
    return matrix

**Example**

In [ ]:
fractionalfactorial2k(3, 1, {3: [1,2]})

array([[-1, -1,  1],
       [ 1, -1, -1],
       [-1,  1, -1],
       [ 1,  1,  1]])

In [ ]:
#| hide
test.test_eq(fractionalfactorial2k(3, 1, {3: [1, 2]}).shape, (4,3))
test.test_eq(
    fractionalfactorial2k(
        8,
        3,
        {
            6: [3, 4, 5],
            7: [1, 2, 4, 5],
            8: [1, 2, 3, 5]
        }).shape, (2**(8-3), 8))

Useful fractional designs according to [NIST](https://www.itl.nist.gov/div898/handbook/pri/section3/pri3347.htm):

In [ ]:
#| export
USEFUL_FRACTIONAL_DESIGNS = {
    (3, 1): {3: [1, 2]},
    (4, 1): {4: [1, 2, 3]},
    (5, 1): {5: [1, 2, 3, 4]},
    (5, 2): {4: [1, 2], 5: [1, 3]},
    (6, 1): {6: [1, 2, 3, 4, 5]},
    (6, 2): {5: [1, 2, 3], 6: [2, 3, 4]},
    (6, 3): {4: [1, 2], 5: [1, 3], 6: [2, 3]},
    (7, 1): {7: [1, 2, 3, 4, 5, 6]},
    (7, 2): {6: [1, 2, 3, 4], 7: [1, 2, 4, 5]},
    (7, 3): {5: [1, 2, 3], 6: [2, 3, 4], 7: [1, 3, 4]},
    (7, 4): {4: [1, 2], 5: [1, 3], 6: [2, 3], 7: [1, 2, 3]},
    (8, 1): {8: [1, 2, 3, 4, 5, 6, 7]},
    (8, 2): {7: [1, 2, 3, 4], 8: [1, 2, 5, 6]},
    (8, 3): {6: [1, 2, 3], 7: [1, 2, 4], 8: [2, 3, 4, 5]},
    (8, 4): {5: [2, 3, 4], 6: [1, 3, 4], 7: [1, 2, 3], 8:[1, 2, 4]},
    (9, 2): {8: [1, 3, 4, 6, 7], 9: [2, 3, 5, 6, 7]},
    (9, 3): {7: [1, 2, 3, 4], 8: [1, 3, 5, 6], 9: [3, 4, 5, 6]},
    (9, 4): {6: [2, 3, 4, 5], 7: [1, 3, 4, 5], 8: [1, 2, 4, 5], 9: [1, 2, 3, 5]},
    (9, 5): {5: [1, 2, 3], 6: [2, 3, 4], 7: [1, 3, 4], 8: [1, 2, 4], 9: [1, 2, 3, 4]},
    (10, 3): {8: [1, 2, 3], 9: [2, 3, 4, 5], 10: [1, 3, 4, 6]},
    (10, 4): {7: [2, 3, 4, 6], 8: [1, 3, 4, 6], 9: [1, 2, 4, 5], 10: [1, 2, 3, 5]},
    (10, 5): {6: [1, 2, 3, 4], 7: [1, 2, 3, 5], 8: [1, 2, 4, 5], 9: [1, 3, 4, 5], 10: [2, 3, 4, 5]},
    (10, 6): {5: [1, 2, 3], 6: [2, 3, 4], 7: [1, 3, 4], 8: [1, 2, 4], 9: [1, 2, 3, 4], 10: [1, 2]},
    (11, 4): {8: [1, 2, 3, 7], 9: [2, 3, 4, 5], 10: [1, 3, 4, 6], 11: [1, 2, 3, 4, 5, 6, 7]},
    (11, 5): {7: [3, 4, 5], 8: [1, 2, 3, 4], 9: [1, 2, 6], 10: [2, 4, 5, 6], 11: [1, 4, 5, 6]},
    (11, 6): {6: [1, 2, 3], 7: [2, 3, 4], 8: [3, 4, 5], 9: [1, 3, 4], 10: [1, 4, 5], 11: [2, 4, 5]},
    (11, 7): {5: [1, 2, 3], 6: [2, 3, 4], 7: [1, 3, 4], 8: [1, 2, 4], 9: [1, 2, 3, 4], 10: [1, 2], 11: [1, 3]},
    (15, 11): {5: [1, 2], 6: [1, 3], 7: [1, 4], 8: [2, 3], 9: [2, 4], 10: [3, 4], 11: [1, 2, 3], 12: [1, 2, 4], 13: [1, 3, 4], 14: [2, 3, 4], 15: [1, 2, 3, 4]}
}

In [ ]:
pprint(USEFUL_FRACTIONAL_DESIGNS)

{(3, 1): {3: [1, 2]},
 (4, 1): {4: [1, 2, 3]},
 (5, 1): {5: [1, 2, 3, 4]},
 (5, 2): {4: [1, 2], 5: [1, 3]},
 (6, 1): {6: [1, 2, 3, 4, 5]},
 (6, 2): {5: [1, 2, 3], 6: [2, 3, 4]},
 (6, 3): {4: [1, 2], 5: [1, 3], 6: [2, 3]},
 (7, 1): {7: [1, 2, 3, 4, 5, 6]},
 (7, 2): {6: [1, 2, 3, 4], 7: [1, 2, 4, 5]},
 (7, 3): {5: [1, 2, 3], 6: [2, 3, 4], 7: [1, 3, 4]},
 (7, 4): {4: [1, 2], 5: [1, 3], 6: [2, 3], 7: [1, 2, 3]},
 (8, 1): {8: [1, 2, 3, 4, 5, 6, 7]},
 (8, 2): {7: [1, 2, 3, 4], 8: [1, 2, 5, 6]},
 (8, 3): {6: [1, 2, 3], 7: [1, 2, 4], 8: [2, 3, 4, 5]},
 (8, 4): {5: [2, 3, 4], 6: [1, 3, 4], 7: [1, 2, 3], 8: [1, 2, 4]},
 (9, 2): {8: [1, 3, 4, 6, 7], 9: [2, 3, 5, 6, 7]},
 (9, 3): {7: [1, 2, 3, 4], 8: [1, 3, 5, 6], 9: [3, 4, 5, 6]},
 (9, 4): {6: [2, 3, 4, 5], 7: [1, 3, 4, 5], 8: [1, 2, 4, 5], 9: [1, 2, 3, 5]},
 (9, 5): {5: [1, 2, 3],
          6: [2, 3, 4],
          7: [1, 3, 4],
          8: [1, 2, 4],
          9: [1, 2, 3, 4]},
 (10, 3): {8: [1, 2, 3], 9: [2, 3, 4, 5], 10: [1, 3, 4, 6]},
 (10, 

In [ ]:
fractionalfactorial2k(6, 2, USEFUL_FRACTIONAL_DESIGNS.get((6, 2)))

array([[-1, -1, -1, -1, -1, -1],
       [ 1, -1, -1, -1,  1, -1],
       [-1,  1, -1, -1,  1,  1],
       [ 1,  1, -1, -1, -1,  1],
       [-1, -1,  1, -1,  1,  1],
       [ 1, -1,  1, -1, -1,  1],
       [-1,  1,  1, -1, -1, -1],
       [ 1,  1,  1, -1,  1, -1],
       [-1, -1, -1,  1, -1,  1],
       [ 1, -1, -1,  1,  1,  1],
       [-1,  1, -1,  1,  1, -1],
       [ 1,  1, -1,  1, -1, -1],
       [-1, -1,  1,  1,  1, -1],
       [ 1, -1,  1,  1, -1, -1],
       [-1,  1,  1,  1, -1,  1],
       [ 1,  1,  1,  1,  1,  1]])

In [ ]:
#| hide
test_6_2 = np.fromstring("""-1  -1  -1  -1  -1  -1
   +1  -1  -1  -1  +1  -1
   -1  +1  -1  -1  +1  +1
   +1  +1  -1  -1  -1  +1
   -1  -1  +1  -1  +1  +1
   +1  -1  +1  -1  -1  +1
   -1  +1  +1  -1  -1  -1
   +1  +1  +1  -1  +1  -1
   -1  -1  -1  +1  -1  +1
   +1  -1  -1  +1  +1  +1
   -1  +1  -1  +1  +1  -1
   +1  +1  -1  +1  -1  -1
   -1  -1  +1  +1  +1  -1
   +1  -1  +1  +1  -1  -1
   -1  +1  +1  +1  -1  +1
   +1  +1  +1  +1  +1  +1""", sep=" ").reshape(-1, 6)

In [ ]:
test.array_equal(
    fractionalfactorial2k(6, 2, USEFUL_FRACTIONAL_DESIGNS.get((6, 2))),
    test_6_2
)

True

In [ ]:
#| hide
test_7_4 = np.fromstring("""-1  -1  -1  +1  +1  +1  -1
   +1  -1  -1  -1  -1  +1  +1
   -1  +1  -1  -1  +1  -1  +1
   +1  +1  -1  +1  -1  -1  -1
   -1  -1  +1  +1  -1  -1  +1
   +1  -1  +1  -1  +1  -1  -1
   -1  +1  +1  -1  -1  +1  -1
   +1  +1  +1  +1  +1  +1  +1""", sep=" ").reshape(-1, 7)
test.array_equal(
    fractionalfactorial2k(7, 4, USEFUL_FRACTIONAL_DESIGNS.get((7, 4))),
    test_7_4
)

True

In [ ]:
#| hide
test_11_4 = np.fromstring("""-1  -1  -1  -1  -1  -1  -1  +1  +1  +1  -1
   +1  -1  -1  -1  -1  -1  -1  -1  +1  -1  +1
   -1  +1  -1  -1  -1  -1  -1  -1  -1  +1  +1
   +1  +1  -1  -1  -1  -1  -1  +1  -1  -1  -1
   -1  -1  +1  -1  -1  -1  -1  -1  -1  -1  +1
   +1  -1  +1  -1  -1  -1  -1  +1  -1  +1  -1
   -1  +1  +1  -1  -1  -1  -1  +1  +1  -1  -1
   +1  +1  +1  -1  -1  -1  -1  -1  +1  +1  +1
   -1  -1  -1  +1  -1  -1  -1  +1  -1  -1  +1
   +1  -1  -1  +1  -1  -1  -1  -1  -1  +1  -1
   -1  +1  -1  +1  -1  -1  -1  -1  +1  -1  -1
   +1  +1  -1  +1  -1  -1  -1  +1  +1  +1  +1
   -1  -1  +1  +1  -1  -1  -1  -1  +1  +1  -1
   +1  -1  +1  +1  -1  -1  -1  +1  +1  -1  +1
   -1  +1  +1  +1  -1  -1  -1  +1  -1  +1  +1
   +1  +1  +1  +1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  +1  -1  -1  +1  -1  +1  +1
   +1  -1  -1  -1  +1  -1  -1  -1  -1  -1  -1
   -1  +1  -1  -1  +1  -1  -1  -1  +1  +1  -1
   +1  +1  -1  -1  +1  -1  -1  +1  +1  -1  +1
   -1  -1  +1  -1  +1  -1  -1  -1  +1  -1  -1
   +1  -1  +1  -1  +1  -1  -1  +1  +1  +1  +1
   -1  +1  +1  -1  +1  -1  -1  +1  -1  -1  +1
   +1  +1  +1  -1  +1  -1  -1  -1  -1  +1  -1
   -1  -1  -1  +1  +1  -1  -1  +1  +1  -1  -1
   +1  -1  -1  +1  +1  -1  -1  -1  +1  +1  +1
   -1  +1  -1  +1  +1  -1  -1  -1  -1  -1  +1
   +1  +1  -1  +1  +1  -1  -1  +1  -1  +1  -1
   -1  -1  +1  +1  +1  -1  -1  -1  -1  +1  +1
   +1  -1  +1  +1  +1  -1  -1  +1  -1  -1  -1
   -1  +1  +1  +1  +1  -1  -1  +1  +1  +1  -1
   +1  +1  +1  +1  +1  -1  -1  -1  +1  -1  +1
   -1  -1  -1  -1  -1  +1  -1  +1  +1  -1  +1
   +1  -1  -1  -1  -1  +1  -1  -1  +1  +1  -1
   -1  +1  -1  -1  -1  +1  -1  -1  -1  -1  -1
   +1  +1  -1  -1  -1  +1  -1  +1  -1  +1  +1
   -1  -1  +1  -1  -1  +1  -1  -1  -1  +1  -1
   +1  -1  +1  -1  -1  +1  -1  +1  -1  -1  +1
   -1  +1  +1  -1  -1  +1  -1  +1  +1  +1  +1
   +1  +1  +1  -1  -1  +1  -1  -1  +1  -1  -1
   -1  -1  -1  +1  -1  +1  -1  +1  -1  +1  -1
   +1  -1  -1  +1  -1  +1  -1  -1  -1  -1  +1
   -1  +1  -1  +1  -1  +1  -1  -1  +1  +1  +1
   +1  +1  -1  +1  -1  +1  -1  +1  +1  -1  -1
   -1  -1  +1  +1  -1  +1  -1  -1  +1  -1  +1
   +1  -1  +1  +1  -1  +1  -1  +1  +1  +1  -1
   -1  +1  +1  +1  -1  +1  -1  +1  -1  -1  -1
   +1  +1  +1  +1  -1  +1  -1  -1  -1  +1  +1
   -1  -1  -1  -1  +1  +1  -1  +1  -1  -1  -1
   +1  -1  -1  -1  +1  +1  -1  -1  -1  +1  +1
   -1  +1  -1  -1  +1  +1  -1  -1  +1  -1  +1
   +1  +1  -1  -1  +1  +1  -1  +1  +1  +1  -1
   -1  -1  +1  -1  +1  +1  -1  -1  +1  +1  +1
   +1  -1  +1  -1  +1  +1  -1  +1  +1  -1  -1
   -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  -1
   +1  +1  +1  -1  +1  +1  -1  -1  -1  -1  +1
   -1  -1  -1  +1  +1  +1  -1  +1  +1  +1  +1
   +1  -1  -1  +1  +1  +1  -1  -1  +1  -1  -1
   -1  +1  -1  +1  +1  +1  -1  -1  -1  +1  -1
   +1  +1  -1  +1  +1  +1  -1  +1  -1  -1  +1
   -1  -1  +1  +1  +1  +1  -1  -1  -1  -1  -1
   +1  -1  +1  +1  +1  +1  -1  +1  -1  +1  +1
   -1  +1  +1  +1  +1  +1  -1  +1  +1  -1  +1
   +1  +1  +1  +1  +1  +1  -1  -1  +1  +1  -1
   -1  -1  -1  -1  -1  -1  +1  -1  +1  +1  +1
   +1  -1  -1  -1  -1  -1  +1  +1  +1  -1  -1
   -1  +1  -1  -1  -1  -1  +1  +1  -1  +1  -1
   +1  +1  -1  -1  -1  -1  +1  -1  -1  -1  +1
   -1  -1  +1  -1  -1  -1  +1  +1  -1  -1  -1
   +1  -1  +1  -1  -1  -1  +1  -1  -1  +1  +1
   -1  +1  +1  -1  -1  -1  +1  -1  +1  -1  +1
   +1  +1  +1  -1  -1  -1  +1  +1  +1  +1  -1
   -1  -1  -1  +1  -1  -1  +1  -1  -1  -1  -1
   +1  -1  -1  +1  -1  -1  +1  +1  -1  +1  +1
   -1  +1  -1  +1  -1  -1  +1  +1  +1  -1  +1
   +1  +1  -1  +1  -1  -1  +1  -1  +1  +1  -1
   -1  -1  +1  +1  -1  -1  +1  +1  +1  +1  +1
   +1  -1  +1  +1  -1  -1  +1  -1  +1  -1  -1
   -1  +1  +1  +1  -1  -1  +1  -1  -1  +1  -1
   +1  +1  +1  +1  -1  -1  +1  +1  -1  -1  +1
   -1  -1  -1  -1  +1  -1  +1  -1  -1  +1  -1
   +1  -1  -1  -1  +1  -1  +1  +1  -1  -1  +1
   -1  +1  -1  -1  +1  -1  +1  +1  +1  +1  +1
   +1  +1  -1  -1  +1  -1  +1  -1  +1  -1  -1
   -1  -1  +1  -1  +1  -1  +1  +1  +1  -1  +1
   +1  -1  +1  -1  +1  -1  +1  -1  +1  +1  -1
   -1  +1  +1  -1  +1  -1  +1  -1  -1  -1  -1
   +1  +1  +1  -1  +1  -1  +1  +1  -1  +1  +1
   -1  -1  -1  +1  +1  -1  +1  -1  +1  -1  +1
   +1  -1  -1  +1  +1  -1  +1  +1  +1  +1  -1
   -1  +1  -1  +1  +1  -1  +1  +1  -1  -1  -1
   +1  +1  -1  +1  +1  -1  +1  -1  -1  +1  +1
   -1  -1  +1  +1  +1  -1  +1  +1  -1  +1  -1
   +1  -1  +1  +1  +1  -1  +1  -1  -1  -1  +1
   -1  +1  +1  +1  +1  -1  +1  -1  +1  +1  +1
   +1  +1  +1  +1  +1  -1  +1  +1  +1  -1  -1
   -1  -1  -1  -1  -1  +1  +1  -1  +1  -1  -1
   +1  -1  -1  -1  -1  +1  +1  +1  +1  +1  +1
   -1  +1  -1  -1  -1  +1  +1  +1  -1  -1  +1
   +1  +1  -1  -1  -1  +1  +1  -1  -1  +1  -1
   -1  -1  +1  -1  -1  +1  +1  +1  -1  +1  +1
   +1  -1  +1  -1  -1  +1  +1  -1  -1  -1  -1
   -1  +1  +1  -1  -1  +1  +1  -1  +1  +1  -1
   +1  +1  +1  -1  -1  +1  +1  +1  +1  -1  +1
   -1  -1  -1  +1  -1  +1  +1  -1  -1  +1  +1
   +1  -1  -1  +1  -1  +1  +1  +1  -1  -1  -1
   -1  +1  -1  +1  -1  +1  +1  +1  +1  +1  -1
   +1  +1  -1  +1  -1  +1  +1  -1  +1  -1  +1
   -1  -1  +1  +1  -1  +1  +1  +1  +1  -1  -1
   +1  -1  +1  +1  -1  +1  +1  -1  +1  +1  +1
   -1  +1  +1  +1  -1  +1  +1  -1  -1  -1  +1
   +1  +1  +1  +1  -1  +1  +1  +1  -1  +1  -1
   -1  -1  -1  -1  +1  +1  +1  -1  -1  -1  +1
   +1  -1  -1  -1  +1  +1  +1  +1  -1  +1  -1
   -1  +1  -1  -1  +1  +1  +1  +1  +1  -1  -1
   +1  +1  -1  -1  +1  +1  +1  -1  +1  +1  +1
   -1  -1  +1  -1  +1  +1  +1  +1  +1  +1  -1
   +1  -1  +1  -1  +1  +1  +1  -1  +1  -1  +1
   -1  +1  +1  -1  +1  +1  +1  -1  -1  +1  +1
   +1  +1  +1  -1  +1  +1  +1  +1  -1  -1  -1
   -1  -1  -1  +1  +1  +1  +1  -1  +1  +1  -1
   +1  -1  -1  +1  +1  +1  +1  +1  +1  -1  +1
   -1  +1  -1  +1  +1  +1  +1  +1  -1  +1  +1
   +1  +1  -1  +1  +1  +1  +1  -1  -1  -1  -1
   -1  -1  +1  +1  +1  +1  +1  +1  -1  -1  +1
   +1  -1  +1  +1  +1  +1  +1  -1  -1  +1  -1
   -1  +1  +1  +1  +1  +1  +1  -1  +1  -1  -1
   +1  +1  +1  +1  +1  +1  +1  +1  +1  +1  +1""", sep=" ").reshape(-1, 11)
test.array_equal(test_11_4, fractionalfactorial2k(11, 4, USEFUL_FRACTIONAL_DESIGNS.get((11, 4))))

True

In [ ]:
test_15_11 = np.fromstring("""-1  -1  -1  -1  +1  +1  +1  +1  +1  +1  -1  -1  -1  -1  +1
   +1  -1  -1  -1  -1  -1  -1  +1  +1  +1  +1  +1  +1  -1  -1
   -1  +1  -1  -1  -1  +1  +1  -1  -1  +1  +1  +1  -1  +1  -1
   +1  +1  -1  -1  +1  -1  -1  -1  -1  +1  -1  -1  +1  +1  +1
   -1  -1  +1  -1  +1  -1  +1  -1  +1  -1  +1  -1  +1  +1  -1
   +1  -1  +1  -1  -1  +1  -1  -1  +1  -1  -1  +1  -1  +1  +1
   -1  +1  +1  -1  -1  -1  +1  +1  -1  -1  -1  +1  +1  -1  +1
   +1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1  -1  -1  -1  -1
   -1  -1  -1  +1  +1  +1  -1  +1  -1  -1  -1  +1  +1  +1  -1
   +1  -1  -1  +1  -1  -1  +1  +1  -1  -1  +1  -1  -1  +1  +1
   -1  +1  -1  +1  -1  +1  -1  -1  +1  -1  +1  -1  +1  -1  +1
   +1  +1  -1  +1  +1  -1  +1  -1  +1  -1  -1  +1  -1  -1  -1
   -1  -1  +1  +1  +1  -1  -1  -1  -1  +1  +1  +1  -1  -1  +1
   +1  -1  +1  +1  -1  +1  +1  -1  -1  +1  -1  -1  +1  -1  -1
   -1  +1  +1  +1  -1  -1  -1  +1  +1  +1  -1  -1  -1  +1  -1
   +1  +1  +1  +1  +1  +1  +1  +1  +1  +1  +1  +1  +1  +1  +1
""", sep=" ").reshape(-1, 15)
test.array_equal(test_15_11, fractionalfactorial2k(15, 11, USEFUL_FRACTIONAL_DESIGNS.get((15,11))))

True

In [ ]:
#| hide
from nbdev import nbdev_export; nbdev_export()